# Capstone Project workbook - Allie Garrison


In [ ]:
import sqlite3
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

%matplotlib inline

In [ ]:
conn = sqlite3.connect('zippedData/im.db')

bom_movie_gross = pd.read_csv("zippedData/bom.movie_gross.csv")
rt_movie_info = pd.read_csv("zippedData/rt.movie_info.tsv", sep='\t')
rt_reviews = pd.read_csv('zippedData/rt.reviews.tsv', sep='\t',encoding='windows-1252')
tmdb_movies = pd.read_csv("zippedData/tmdb.movies.csv")
tn_movie_budgets = pd.read_csv("zippedData/tn.movie_budgets.csv", parse_dates=['release_date'])

In [ ]:
rt_reviews.head()

In [ ]:
rt_reviews.info()

In [ ]:
bom_movie_gross.info()

In [ ]:
bom_movie_gross.head()

In [ ]:
rt_movie_info.info()

In [ ]:
rt_movie_info.head()

In [ ]:
tmdb_movies.info()

In [ ]:
tmdb_movies.head()

In [ ]:
tn_movie_budgets.info()

In [ ]:
tn_movie_budgets.head()

In [ ]:
tn_movie_budgets.plot.bar('production_budget')

In [ ]:
## Making copy of movie budgets data frame for cleaning
tn_movie_budgets_clean = tn_movie_budgets.copy()

## turning the columns with dollar amounts into int types in millions of dollars
tn_movie_budgets_clean['production_budget_in_millions'] = tn_movie_budgets_clean['production_budget'].str.strip('$').str.replace(',', '')
tn_movie_budgets_clean['production_budget_in_millions'] = tn_movie_budgets_clean['production_budget_in_millions'].astype('int64') / 1000000

tn_movie_budgets_clean['domestic_gross_in_millions'] = tn_movie_budgets_clean['domestic_gross'].str.strip('$').str.replace(',', '')
tn_movie_budgets_clean['domestic_gross_in_millions'] = tn_movie_budgets_clean['domestic_gross_in_millions'].astype('int64') / 1000000

tn_movie_budgets_clean['worldwide_gross_in_millions'] = tn_movie_budgets_clean['worldwide_gross'].str.strip('$').str.replace(',', '')
tn_movie_budgets_clean['worldwide_gross_in_millions'] = tn_movie_budgets_clean['worldwide_gross_in_millions'].astype('int64') / 1000000

## keeping only the movies released after January 1, 2010
tn_movie_budgets_clean = tn_movie_budgets_clean[tn_movie_budgets_clean['release_date'] >= "2010-01-01"]

## removing all the movies that made $0
tn_movie_budgets_clean = tn_movie_budgets_clean[tn_movie_budgets_clean['domestic_gross_in_millions'] != 0]

In [ ]:
## Calculating return on investment (ROI) for domestic and international revenue
tn_movie_budgets_clean['ROI_domestic'] = (tn_movie_budgets_clean['domestic_gross_in_millions'] / tn_movie_budgets_clean['production_budget_in_millions']) * 100
tn_movie_budgets_clean['ROI_international'] = (tn_movie_budgets_clean['worldwide_gross_in_millions'] / tn_movie_budgets_clean['production_budget_in_millions']) * 100
tn_movie_budgets_clean.head()

In [ ]:
q = '''
SELECT *
FROM persons
LEFT JOIN known_for k USING (person_id)
LEFT JOIN movie_basics m USING (movie_id)

'''

pd.read_sql(q, conn)

In [ ]:
pd.read_sql('''
SELECT  genres, COUNT() AS count_of_genres
FROM movie_basics
GROUP BY genres
ORDER BY count_of_genres DESC
''', conn)

In [ ]:
table3 = pd.read_sql('''
SELECT  title
FROM movie_akas
WHERE is_original_title = 1

''', conn)

### 

In [ ]:
## Joining the cleaned budget dataframe onto a queried dataframe from sql database on movie title
budget_w_sql = tn_movie_budgets_clean.merge(table3, how='inner', left_on='movie', right_on='title')

In [ ]:
budget_w_sql.describe()

In [ ]:
table_genre = pd.read_sql('''
SELECT mb.primary_title, 
        mb.runtime_minutes, 
        mb.genres, 
        mr.averagerating,
        mr.numvotes
FROM movie_basics AS mb
JOIN movie_ratings AS mr USING (movie_id)
''', conn)

In [ ]:
genre_ratings_budget = tn_movie_budgets_clean.merge(table_genre, how='inner', left_on='movie', right_on='primary_title')

In [ ]:
genre_ratings_budget.head(15)

In [ ]:
genre_ratings_budget.describe()

In [ ]:
genre_ratings_budget.runtime_minutes.plot.box()

In [ ]:
action = genre_ratings_budget[genre_ratings_budget['genres'].str.contains('Drama') == True]
avg_rating = genre_ratings_budget['averagerating'].mean()
avg_rating, action['averagerating'].mean()

In [ ]:
stats.ttest_1samp(action['averagerating'], popmean=avg_rating)

In [ ]:
genre_ratings_budget.genres.describe()

In [ ]:
genre_ratings_budget['profit_in_millions'] = genre_ratings_budget['worldwide_gross_in_millions'] - genre_ratings_budget['production_budget_in_millions']

In [ ]:
# genre_hash is a dictionary where each key is a genre, and the value is the number of times it occurs in the 'genres' column
genre_hash = {}

# array of lists where each individual list is every genre listed for a single movie
genre_lists = genre_ratings_budget['genres'].str.split(',')

for i, genres in enumerate(genre_lists):
    # try/ except in case there is an empty list (no genres listed for a movie)
    try:
        for genre in genres:
            # add one to genre if the key exists
            if genre in genre_hash.keys():
                genre_hash[genre] += 1
            # otherwise create the key
            else:
                genre_hash[genre] = 1
    except:
        pass

In [ ]:
genre_stats = []
for genre in genre_hash.keys():
    #filter for every entry that contains the current genre
    mask = genre_ratings_budget[genre_ratings_budget['genres'].str.contains(genre) == True]
    # append list with averages for relevant features
    genre_stats.append([genre, mask['profit_in_millions'].mean().round(2), 
                        mask['production_budget_in_millions'].mean().round(2),
                        mask['averagerating'].mean().round(2),
                        mask['numvotes'].mean().round(2)])

In [ ]:
#convert to dataframe
genre_stats_df = pd.DataFrame(genre_stats, 
                              columns=['Genre', 'Average_Profit', 'Average_Budget', 'Average_Rating', 'Average_Votes'])
genre_stats_df

In [ ]:
action = genre_ratings_budget[genre_ratings_budget['genres'].str.contains('Animation') == True]
avg_rating = genre_ratings_budget['profit_in_millions'].mean()
avg_rating, action['profit_in_millions'].mean()

In [ ]:
stats.ttest_1samp(action['profit_in_millions'], popmean=avg_rating)

In [ ]:
animation = genre_ratings_budget[genre_ratings_budget['genres'].str.contains('Animation') == True]

In [ ]:
animation['runtime_bins'] = animation['runtime_minutes'].apply(bin_runtime)

In [ ]:
def bin_runtime(x):
    if x >= 110:
        return '110-120'
    if x < 110 and x >= 100:
        return '100-109'
    if x < 100 and x >= 90:
        return '90-99'
    elif x < 90 and x >= 80:
        return '80-89'
    elif x < 80 and x >= 70:
        return '70-79'
    elif x < 70 and x >= 60:
        return '60-69'
  

In [ ]:
animation.runtime_bins.isna().count()

In [ ]:
animation.runtime_minutes.describe()